In [1]:
import cv2
import numpy as np
import math
import time
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib
import torch
from torchvision import transforms
import util
from torchsummary import summary
from body import Body
import import_ipynb
from train_model import BodyposeBackbone, MainModel
from coco_dataset import COCODataset, transform_image
from copy import deepcopy
import random
import pandas as pd
from tqdm import tqdm
import sys
import os

/tmp/ipykernel_13821/2007558362.py:5: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


importing Jupyter notebook from train_model.ipynb
importing Jupyter notebook from coco_dataset.ipynb


/home/jakub/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
NO_OF_OUTPUTS = 17

In [3]:
colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
            [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
            [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85], [255, 255, 255]]

In [4]:
new_order = [0, 15, 14, 17, 16, 5, 2, 6, 3, 7, 4, 11, 8, 12, 9, 13, 10]

In [5]:
backbone_model_path = '../model/body_pose_model.pth'

In [6]:
backbone_model = BodyposeBackbone()

In [7]:
backbone_params = torch.load(backbone_model_path)

In [8]:
backbone_dict = util.transfer(backbone_model, backbone_params)

In [9]:
backbone_model.load_state_dict(backbone_dict)

<All keys matched successfully>

In [10]:
main_model = MainModel(backbone_model, NO_OF_OUTPUTS)

Create dataset from COCO .json file

In [11]:
train_data = COCODataset('/media/jakub/One Touch/coco_pose/coco2017labels-pose/coco-pose', transforms=transform_image(train=True))

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


In [12]:
skeleton = train_data.coco.cats[1]['skeleton']
skeleton = [[connection[0]-1, connection[1]-1] for connection in skeleton]

In [13]:
index = random.randint(0, 100)
anns = []
while anns == []:
    for id in train_data.coco.getAnnIds():
        if train_data.coco.anns[id]['image_id'] == train_data.coco.getImgIds()[index]:
            ann = train_data.coco.anns[id]
            if 1 in ann['keypoints'] or 2 in ann['keypoints']:
                anns.append(ann)
    if anns == []:
        index = random.randint(0, 100)

In [14]:
bboxes = [ann['bbox'] for ann in anns]

In [15]:
bboxes = [[int(element) for element in box] for box in bboxes]

In [16]:
keypoints = [ann['keypoints'] for ann in anns]

In [17]:
new_keypoints = []
for keypoint in keypoints:
    new_keypoints.append([[keypoint[i], keypoint[i+1], keypoint[i+2]] for i in range(0, len(keypoint), 3)])
keypoints = new_keypoints

Show example image

In [18]:
image = train_data._load_image(train_data.coco.getImgIds()[index])
for bbox in bboxes:
    cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[0]+bbox[2], bbox[1]+bbox[3]), (0, 0, 255))
for i, connection in enumerate(skeleton):
    for keypoint in keypoints:
        point1 = keypoint[connection[0]]
        point2 = keypoint[connection[1]]
        if point1[2] != 0 and point2[2] != 0:
            cv2.line(image, point1[:-1], point2[:-1], color=colors[i], thickness=3)
cv2.imshow('Example image', image)
cv2.waitKey(5_000)
cv2.destroyAllWindows()

In [19]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [20]:
device = torch.device("cpu")

In [22]:
images,targets = train_data[index]

In [23]:
output = main_model(images)

In [24]:
loss = torch.nn.MSELoss()
params = [p for p in main_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params)

In [25]:
def train(main_model, loss, optimizer, inputs, outputs):
    optimizer.zero_grad()

    main_model.train()

    out1 = backbone_model(inputs)
    logits = main_model(out1)
    output = loss.forward(logits, outputs)
    output.backward()
    optimizer.step()

    return output.item()